In [20]:
# !pip install datasetsforecast
# !pip install hierarchicalforecast
# !pip install statsforecast

In [21]:
import numpy as np
import pandas as pd
import os


from datasetsforecast.hierarchical import HierarchicalData
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.methods import  BottomUp, TopDown, MiddleOut, MinTrace, ERM
from statsforecast.core import StatsForecast
from statsforecast.models import AutoARIMA, Naive
from hierarchicalforecast.evaluation import HierarchicalEvaluation

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import root_mean_squared_error as rmse

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight') 

In [22]:
# general settings
class CFG:
    data_folder = '../SAP_AI/Demand Forecasting/Datasets/'
    img_dim1 = 20
    img_dim2 = 10

# adjust the parameters for displayed figures    
plt.rcParams.update({'figure.figsize': (CFG.img_dim1,CFG.img_dim2)})   

In [23]:
def my_rmse(x,y):
    return(np.round( np.sqrt(mse(x.values,y.values)) ,4))

In [24]:
# sales data calendar_df = pd.read_csv('../input/m5-forecasting-accuracy/calendar.csv', parse_dates=['date'])
file_path = '/home/user/projects/SAP_AI/Demand Forecasting/Datasets/'
calendar_df = pd.read_csv(file_path + 'calendar.csv', parse_dates=['date'])
calendar_df = calendar_df.loc[:, ['date', 'wm_yr_wk', 'd']]
df = pd.read_csv(file_path + 'sales_train_evaluation.csv')
df = df.loc[df.item_id=='FOODS_3_819']
df_T = df.melt(id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])
df_T.drop(columns=['id'], inplace=True)

sales_df = df_T.merge(calendar_df, left_on='variable', right_on='d', how='left')
sales_df.rename(columns={'value': 'sales_qty'}, inplace=True)
df = sales_df.loc[sales_df.date >= '2014-01-01', ['date', 'store_id', 'sales_qty']]
df['state_id'] = df['store_id'].str[:2]

df.head(3)

,date,store_id,sales_qty,state_id
10680,2014-01-01,CA_1,3,CA
10681,2014-01-01,CA_2,0,CA
10682,2014-01-01,CA_3,7,CA


In [25]:
# Ensure that 'date' column is in datetime format
df['date'] = pd.to_datetime(df['date'])

# Add five years using DateOffset
df['date'] = df['date'] + pd.DateOffset(years=8)

# Display the updated DataFrame
print(df)

            date store_id  sales_qty state_id
10680 2022-01-01     CA_1          3       CA
10681 2022-01-01     CA_2          0       CA
10682 2022-01-01     CA_3          7       CA
10683 2022-01-01     CA_4          0       CA
10684 2022-01-01     TX_1          0       TX
...          ...      ...        ...      ...
19405 2024-05-22     TX_2          0       TX
19406 2024-05-22     TX_3          1       TX
19407 2024-05-22     WI_1          7       WI
19408 2024-05-22     WI_2          0       WI
19409 2024-05-22     WI_3          1       WI

[8730 rows x 4 columns]


In [26]:
# create the long format matrix: individual stores
df_ind = df.groupby(['date', 'store_id'])[['sales_qty']].sum()
df_ind.reset_index(inplace=True)
df_ind = df_ind.T.reset_index(drop=True).T
df_ind.columns = ['ds', 'unique_id', 'sales']

# create the long format matrix: state level
df_sta = df.groupby(['date', 'state_id'])[['sales_qty']].sum()
df_sta.reset_index(inplace=True)
df_sta.columns = ['ds', 'unique_id', 'sales']

# create the long format matrix: total level
df_tot = df.groupby(['date'])[['sales_qty']].sum()
df_tot.reset_index(inplace=True)
df_tot['unique_id'] = 'Total'
df_tot.columns = ['ds', 'sales', 'unique_id' ]


# combine all three
dfx = pd.concat([df_ind, df_sta, df_tot], axis = 0)
print(df_ind.shape, df_sta.shape, df_tot.shape, dfx.shape)

# format
xset = set(dfx.unique_id)
dfx.columns = ['ds','unique_id', 'y']
dfx['ds'] = pd.to_datetime(dfx['ds'])
dfx.head(10)

(8730, 3) (2619, 3) (873, 3) (12222, 3)


,ds,unique_id,y
0,2022-01-01,CA_1,3
1,2022-01-01,CA_2,0
2,2022-01-01,CA_3,7
3,2022-01-01,CA_4,0
4,2022-01-01,TX_1,0
5,2022-01-01,TX_2,1
6,2022-01-01,TX_3,1
7,2022-01-01,WI_1,1
8,2022-01-01,WI_2,0
9,2022-01-01,WI_3,3


In [27]:
S = np.zeros((len(xset), len([f for f in xset if '_' in f])))


# rows / columns
list1 = ['Total', 'CA','CA_1','CA_2','CA_3','CA_4','TX','TX_1','TX_2','TX_3','WI','WI_1','WI_2','WI_3']
list2 = ['CA_1','CA_2','CA_3','CA_4','TX_1','TX_2','TX_3','WI_1','WI_2','WI_3']
S = pd.DataFrame(S); S.index = list1; S.columns = list2


# encode the hierarchical structure
S.loc['Total'] = 1
S.loc['CA'][['CA_1','CA_2','CA_3', 'CA_4']] = 1
S.loc['TX'][['TX_1','TX_2','TX_3']] = 1
S.loc['WI'][['WI_1','WI_2','WI_3']] = 1
for x in S.columns:
    S.loc[x][x]= 1
S = S.astype(int)
S

/tmp/ipykernel_21447/4040101078.py:12: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  S.loc['CA'][['CA_1','CA_2','CA_3', 'CA_4']] = 1
/tmp/ipykernel_21447/4040101078.py:13: FutureWarning: ChainedAssignmentError: behaviour will change in panda

,CA_1,CA_2,CA_3,CA_4,TX_1,TX_2,TX_3,WI_1,WI_2,WI_3
Total,1,1,1,1,1,1,1,1,1,1
CA,1,1,1,1,0,0,0,0,0,0
CA_1,1,0,0,0,0,0,0,0,0,0
CA_2,0,1,0,0,0,0,0,0,0,0
CA_3,0,0,1,0,0,0,0,0,0,0
CA_4,0,0,0,1,0,0,0,0,0,0
TX,0,0,0,0,1,1,1,0,0,0
TX_1,0,0,0,0,1,0,0,0,0,0
TX_2,0,0,0,0,0,1,0,0,0,0
TX_3,0,0,0,0,0,0,1,0,0,0


In [28]:
tags = {}
tags['Country'] = np.array(['Total'], dtype=object)
tags['Country/State'] = np.array(['CA', 'TX', 'WI'], dtype=object)
tags['Country/State/Store'] = np.array(['CA_1', 'CA_2', 'CA_3', 'CA_4',  
                                        'TX_1', 'TX_2', 'TX_3',
                                        'WI_1', 'WI_2', 'WI_3'], dtype=object)
tags

{'Country': array(['Total'], dtype=object),
 'Country/State': array(['CA', 'TX', 'WI'], dtype=object),
 'Country/State/Store': array(['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1',
        'WI_2', 'WI_3'], dtype=object)}

In [29]:
horizon = 7 

x_test = dfx.groupby('unique_id').tail(horizon)
x_train = dfx.drop(x_test.index)
x_test = x_test.set_index('unique_id')
x_train = x_train.set_index('unique_id')

In [30]:
# Check data types of the DataFrame
print(x_train.dtypes)

# Ensure the target column 'y' is numeric
x_train['y'] = pd.to_numeric(x_train['y'], errors='coerce')

# Check if there are any NaN values introduced by coercion
print(x_train['y'].isna().sum())

# Drop or handle NaN values if any were introduced
x_train = x_train.dropna(subset=['y'])

ds    datetime64[ns]
y             object
dtype: object
0


In [12]:
# Compute base auto-ARIMA predictions
fcst = StatsForecast(df = x_train, models=[AutoARIMA(season_length= 7)], freq='D', n_jobs=-1)
x_hat = fcst.forecast(h = horizon)

/home/user/projects/SAP_AI/env/lib/python3.11/site-packages/statsforecast/core.py:467: FutureWarning: The `df` argument of the StatsForecast constructor as well as reusing stored dfs from other methods is deprecated and will raise an error in a future version. Please provide the `df` argument to the corresponding method instead, e.g. fit/forecast.
  warnings.warn(
/home/user/projects/SAP_AI/env/lib/python3.11/site-packages/statsforecast/core.py:619: FutureWarning: Passing unique_id as the index is deprecated. Please provide it as a column instead.
  warnings.warn(
/home/user/projects/SAP_AI/env/lib/python3.11/site-packages/statsforecast/core.py:467: FutureWarning: The `df` argument of the StatsForecast constructor as well as reusing stored dfs from other methods is deprecated and will raise an error in a future version. Please provide the `df` argument to the corresponding method instead, e.g. fit/forecast.
  warnings.warn(
/home/user/projects/SAP_AI/env/lib/python3.11/site-packages/st

In [13]:
x_hat

,ds,AutoARIMA
unique_id,,
CA,2024-05-16,6.236943
CA,2024-05-17,6.033696
CA,2024-05-18,6.033696
CA,2024-05-19,6.033696
CA,2024-05-20,6.033696
...,...,...
WI_3,2024-05-18,1.885268
WI_3,2024-05-19,1.885268
WI_3,2024-05-20,1.885268


In [14]:
xmat = pd.merge(left = x_test, right = x_hat, on = ['ds', 'unique_id'])
xmat.head(3)

,ds,y,AutoARIMA
unique_id,,,
CA_1,2024-05-16,2,1.847775
CA_2,2024-05-16,0,1.547379
CA_3,2024-05-16,1,1.699377


In [15]:
xmat = pd.merge(left = x_test, right = x_hat, on = ['ds', 'unique_id'])
xmat.columns = [['ds', 'y', 'pred']]
print('overall rmse: ' + str(my_rmse(xmat['y'], xmat['pred'])))
for k in tags.keys():
    print(k + ' rmse: ' + str(my_rmse(xmat.loc[tags[k]]['y'], xmat.loc[tags[k]]['pred'])))

overall rmse: 2.2602
Country rmse: 4.8227
Country/State rmse: 2.8444
Country/State/Store rmse: 1.5488


Bottom Up

In [16]:
# Reconcile the base predictions
reconcilers = [
    BottomUp()
]

hrec = HierarchicalReconciliation(reconcilers = reconcilers)
#x_hat_rec = hrec.reconcile(x_hat, x_train, S, tags)
x_hat_rec = hrec.reconcile(x_hat, S, tags)

In [17]:
def mse(y, y_hat):
    return np.mean((y-y_hat)**2)

evaluator = HierarchicalEvaluation(evaluators=[mse])
evaluator.evaluate(Y_hat_df = x_hat_rec, Y_test_df = x_test, 
                   tags=tags, benchmark='AutoARIMA')

,,AutoARIMA,AutoARIMA/BottomUp
level,metric,,
Overall,mse-scaled,1.0,0.939187
Country,mse-scaled,1.0,1.110586
Country/State,mse-scaled,1.0,0.714851
Country/State/Store,mse-scaled,1.0,1.0


In [18]:
xmat = pd.merge(left = x_test, right = x_hat_rec, on = ['ds', 'unique_id'])
xmat.head(3)

,ds,y,AutoARIMA,AutoARIMA/BottomUp
unique_id,,,,
CA_1,2024-05-16,2,1.847775,1.847775
CA_2,2024-05-16,0,1.547379,1.547379
CA_3,2024-05-16,1,1.699377,1.699377


In [19]:
xmat = pd.merge(left = x_test, right = x_hat_rec, on = ['ds', 'unique_id'])
xmat.columns = [['ds', 'y', 'pred_orig', 'pred_reconciled']]
mse1 = my_rmse(xmat['y'], xmat['pred_orig'])
mse2 = my_rmse(xmat['y'], xmat['pred_reconciled'])
print('overall rmse: ' + str(mse1) + ' -> ' + str(mse2))

for k in tags.keys():
    mse1 = my_rmse(xmat.loc[tags[k]]['y'], xmat.loc[tags[k]]['pred_orig'])
    mse2 = my_rmse(xmat.loc[tags[k]]['y'], xmat.loc[tags[k]]['pred_reconciled'])

    print(k + ' rmse: ' + str(mse1) + ' -> ' + str(mse2) )

overall rmse: 2.2602 -> 2.1904
Country rmse: 4.8227 -> 5.0823
Country/State rmse: 2.8444 -> 2.4049
Country/State/Store rmse: 1.5488 -> 1.5488
